# Histograms as Objects: Tools for Efficient Analysis and Interactivity

### Aman Goel (speaker), Dr. Hans Peter Dembinski, Henry Fredrick Schreiner, Jay Gohil (speaker)

**12th September, 2022**

- **Overview**
    - boost-histogram
    - hist
    - uproot-browser

- **Quick Links**
    - Talk - [Indico](https://indico.cern.ch/event/1150631/contributions/5014273/), [Repository](https://github.com/amangoel185/PyHEP2022-histogramming), [Zenodo](), [Binder]()
    - boost-histogram - [Documetation](https://boost-histogram.readthedocs.io), [Repository](https://github.com/scikit-hep/boost-histogram)
    - hist - [Documentation](https://hist.readthedocs.io), [Repository](https://github.com/scikit-hep/hist)
    - uproot-browser - [Repository](https://github.com/scikit-hep/uproot-browser)

- **Ideas and structure**
    - via Jay
        - Histogram fundamentals
        - Quickstart and tutorial
        - Axes
        - Storage
        - Accumulators
        - Indexing
        - Plotting
        - Transformations
    - what pattern for talk?
        - boost-histogram first, then add on with features from Hist?
        - then present uproot-browser
        - complelety in noteboook or with slides? use extension in notebook for presentation?
        - make one histogram and use throughout boost-histogram, hist and uproot-browser?
        - image based or example based demo?
    - miscellaneous
        - how to generate repository like Henry's?
        - releases and CI?